## Import needed libraries

In [ ]:
!!pip freeze # shows you all the packages installed in your local environment

In [1]:
import json
import requests
import time

## Assign variables to Variables 

These will be used later in the code. In the future, we may want to import these variables from a .yaml or .json file. Configurations should be imported or extracted from a parsable, human-friendly config file. When setting up complex systems, it's nice to have configuration files thoughtfully organized.

First let's define some variables that will be helpful.  These values are stored in `food-app-database/instance/config.py`

*NDBNO_TOTAL* is the last recorded number of items in the USDA database.  

*LAST_SR* is the version number of the USDA Database

*API_KEY* is the identifier acquired through USDA api services

*q* is the search term (any string) for the search API.  We'll most likely leave this blank.

*ds* is the datasource.  Must be 'Branded Food Products', 'Standard Reference', or ''.  We'll leave it blank to include both.

*fg* is the Food group ID.  We'll also leave this blank.

*sort* the results by food name (n) or by search relevance (r).  We'll sort by food name (n) for standardization's sake.

*mx* refers to the maximum number of items to return.  This seems to max out at 1500.

*offset* determines the index of the beginning row in the results set to begin.  

*formt* can either be JSON ('json') or XML ('xml).  We'll stick with JSON.

In [21]:
LAST_NDBNO_TOTAL = 200000
LAST_SR = 28
current_ndbo_total = 0
current_sr = 0
API_KEY = '7WqOHQdC2shEfBrx25bIEwxBkvUkYTHMoHYlLWL8' #1000 requests/hour
q = ''
ds = ''
fg = ''
sort = 'n'
mx = 35 # max is 1500 
offset = 0
formt = 'json' 
q = '' 
typ = 'f'
ndbno_id = 0


# special api request to get meta information on database (total number of items, standard reference version)
initial_search_request = 'https://api.nal.usda.gov/ndb/search/?format=json&q=&sort=n&max=1&offset=0&api_key=7WqOHQdC2shEfBrx25bIEwxBkvUkYTHMoHYlLWL8'

# search API
search_api_request = 'https://api.nal.usda.gov/ndb/search/?format={}&q={}&sort={}&max={}&offset={}&api_key={}'.format(formt, q, sort, mx, offset, API_KEY)

# Get JSON report with API
report_api_request = 'https://api.nal.usda.gov/ndb/reports/?ndbno={}&type={}&format={}&api_key={}'.format(ndbno_id, typ, formt, API_KEY)

## Let's define some useful functions 

In [22]:
def get_db_status(initial_search_request):
    '''
    initial_search_request = 'https://api.nal.usda.gov/ndb/search/?format=json&q=&sort=n&max=1&offset=0&api_key=DEMO_KEY'
    current_total = total number of items in usda database at the time of request
    current_sr = Standard Release version of the data at the time of request
    
    Returns {'current_total': current_total, 'current_sr': current_sr}
    '''
    usda_database_check = requests.get(initial_search_request)
    check_json = usda_database_check.json()
    current_total = check_json['list']['total']
    current_sr = check_json['list']['sr']
    return {'current_total': current_total, 'current_sr': current_sr}

def check_dbitem_changed(LAST_NDBNO_TOTAL, current_ndbno_total, LAST_SR, current_sr): #TODO: convert to decorator
    '''Returns True if Database has a different number of items
    '''
    if LAST_NDBNO_TOTAL != current_ndbno_total:
        return True
    elif LAST_SR != current_sr:
        return True
    else:
        return False

def get_ndbno_list(search_api_request, formt, q, sort, mx, offset, API_KEY):
    '''This returns a list of ndbno's in the usda foods database
    '''
    search_object = requests.get(search_api_request) 
    search_json = search_object.json() # convert search_object to JSON

    ndbno_list_dict = search_json['list']['item'] # ndbno_list_dict is a list of dictionaries, where each dictionary is a unique food item
    
    return ndbno_list_dict

def get_ndbno_full_report(report_api_request, ndbno_id):
    '''Returns JSON Full Report 
    '''
    full_report = requests.get(report_api_request)
    full_report_json = full_report.json()
    return full_report_json
    

## Determine the number of items in the USDA database 

We will be using this marker as well as the lastupdated date to initiate a scan for new data.  running the webiste through internetarchives will do the trick.

In [23]:
db_status = get_db_status(initial_search_request)
current_ndbno_total = db_status['current_total']
current_sr = db_status['current_sr']
print("Current Number of Items in database: ", current_ndbno_total)
print("Current Standard Reference Database Version: ", current_sr)

Current Number of Items in database:  214072
Current Standard Reference Database Version:  28


## Let's see if there's been an update in the USDA database

In [24]:
print(check_dbitem_changed(LAST_NDBNO_TOTAL, current_ndbno_total, LAST_SR, current_sr))

True


## If there's been an update, let's get a list of all the ndbno's in the database at this current time

In [26]:
# total amount that we can search 
print("The maximum number of items we can pull from a request is: ", mx)

# determine how many number of API request for searches will need to be done
import math
# search_num = math.ceil(current_ndbno_total/mx) # to round up!
search_num = 1 # for testing

# for each search, we need to keep track of the offset counter
offset_counter = 0

ndbno_list = []

for int in range(0, search_num):
    # determine offset
    offset_temp = offset + (offset_counter * mx) 
    
    temp_ndbno_list = get_ndbno_list(search_api_request, formt, q, sort, mx, offset_temp, API_KEY)
    
    for item in temp_ndbno_list:
        ndbno_list.append(item)
        
    # increase offset counter
    offset_counter += 1

# print results
for item in ndbno_list:
    print(item['name'])

The maximum number of items we can pull from a request is:  35
AARDVARK HABENERO HOT SAUCE, UPC: 853393000030
AARON'S BEST, OVEN ROASTED TURKEY BREAST, UPC: 783919440012
A&B AMERICAN STYLE, MORE HEAT SMALL BATCH PEPPER SAUCE, UPC: 851063005040
A&B AMERICAN STYLE, ORGANIC PEPPER SAUCE, UPC: 851063005118
A&B AMERICAN STYLE, PEPPER SAUCE, GARLIC, UPC: 851063005064
A&B AMERICAN STYLE, SMALL BATCH PEPPER SAUCE, ORIGINAL, UPC: 851063005019
A. BAUER'S, PREPARED MUSTARD, UPC: 723738002022
ABBA-ZABA, SNACK SIZE BITES CANDY, UPC: 070602609000
ABBA-ZABA'S, TAFFY, WILD STRAWBERRY, SOUR, UPC: 070602066001
ABBEY FARM, RHUBARB & GINGER PRESERVE, UPC: 763242039022
ABBOTSFORD FARMS, CAGE FREE 100% LIQUID EGG WHITES, UPC: 014616221100
ABBOTT, EAS, ADVANTEDGE CARB CONTROL SHAKES, CAFE CARAMEL, UPC: 791083309240
ABBOTT, EAS, ADVANT EDGE PURE MILK PROTEIN SHAKE, DELUXE DARK CHOCOLATE, UPC: 791083304368
ABBOTT, EAS, MYOPLEX 30 BUILD MUSCLE BAR, CHOCOLATE PEANUT BUTTER, UPC: 791083622813
ABBOTT, EAS, PURE WH

create database instances for food_desc, food_unit, food_upc for each item in pull request

In [ ]:
db.create_all()

# let's unpack name and UPC code from item['name']
name, upc = item['name'].split() # how to do this?

# 



## Let's crossreference this with all the ndbno's in the USDA food's database

There's should be a better way to do this without downloading all the ndbno's each time...

## Now let's get a list of all the new ndbno's

Let's fetch the JSON data of each of the new ndbno's

In [ ]:
type = 'f' # get full reports only

usda_database_check = requests.get(initial_search_request)
check_json = usda_database_check.json()

print(check_json)

items_total = check_json['list']['total'] # total number of species
print(items_total)

# get all ndbno in database
get_all_items_api_request = 'https://api.nal.usda.gov/ndb/search/?format=json&q=&sort=n&max=1&offset=0&api_key=DEMO_KEY'

In [ ]:
example = requests.get(example_food_comp_api_request)
print(type(example), example)

In [ ]:
example_json = example.json()

print(type(example_json), example_json)

In [ ]:
start_url = "https://ndb.nal.usda.gov/ndb/search/list"
api_key1 = '7WqOHQdC2shEfBrx25bIEwxBkvUkYTHMoHYlLWL8'
api_key2 = 'avWkKpCJmnTjviy9vfKmsaZkGALXNabV05zUyXQ1'

example_search_api_request = 'https://api.nal.usda.gov/ndb/search/?format=json&q=butter&max=25&offset=0&api_key=DEMO_KEY'
example_food_comp_api_request = 'https://api.nal.usda.gov/ndb/reports/?ndbno=01009&type=f&format=xml&api_key=DEMO_KEY'
                                
                              

html_target = "a"
tag = "href"
f_ext = ".csv"
dir_name = "..."

### Request and Collect

We instantiate a request object and call the `.get` method on it. `r` is our `HTTP 1.1` response. 

From here we have:

*    status
*    encoding
*    text of the body --- should type check this
*    content of the body --- type binary

Once we have our `html` we are ready to scrape the site for useful `href` tags`



In [ ]:
result = requests.get(start_url)

status = result.status_code
encoding = result.encoding
html_doc = result.text
c = result.content

print(status, encoding)

Now let's create a beautifulsoup object out of the request object so we can more easily navigate the html

In [ ]:
soup = BeautifulSoup(c, 'lxml')

In [ ]:
print(soup.prettify())

In [ ]:
soup.title

In [ ]:
soup.title.name

In [ ]:
soup.title.string

In [ ]:
soup.title.parent.name

In [ ]:
soup.p

In [ ]:
soup.a

#### download_url(url, endpoint)

This function makes a new request each time it's called. It writes the binary content to file.
This could be two functions:
1. Get the new request object/content 
2. Write that content to file 

In [ ]:
def download_url(url, endpoint):
    """
    return: None
    downloads file, requires url in global or class scope.
    """
    url_addr = "{url}/{endpoint}".format(url=url, endpoint=endpoint)
    file_path = "{directory}/{endpoint}".format(directory=dir_name, endpoint=endpoint)
    
    r = requests.get(url_addr)
    content_file = r.content
    
    with open(file_path, 'wb') as f:
        print """Downloading From: {url}\nWriting to: {file_path}""".format(
                                                url=url_addr, 
                                                file_path=file_path
                                                                    )
        f.write(content_file)